# Working with a decomposition tree
In this notebook we will show how to get and extract insights from a decomposition tree, a structure returned by some HTN planners (we will use SIADEX here) that gives information about how tasks should be decomposed, starting from the initial task-goal, to achieve the resulting plan.

For more information on hierarchical planning, and what tasks and method are, take a look at the [SIADEX](siadex.ipynb) notebook or the [Wikipedia page](https://en.wikipedia.org/wiki/Hierarchical_task_network).

First, we will need to install SIADEX and register it as an UPF engine

####  _______ Begin Installation____________

**Disclaimer** : The installation steps are only needed until up_siadex is published on pypi and the changes on the unified-planning fork are merged into the origin package

In [ ]:
# Make sure the following packages are installed on the system: python-dev libreadline-dev. Those are needed for the execution of Siadex
!apt-get update
!apt-get install -y python-dev libreadline-dev

In [ ]:
# Clonning the repos
!git clone https://github.com/UGR-IntelligentSystemsGroup/unified-planning.git
!git clone https://github.com/UGR-IntelligentSystemsGroup/up-siadex.git

In [ ]:
# Install the packages
# %%capture
%pip install ./unified-planning
%pip install ./up-siadex

####  _______ End Installation____________

In [1]:
from up_siadex import SIADEXEngine

import unified_planning as up
from unified_planning.shortcuts import *
from unified_planning.model.htn.hierarchical_problem import HierarchicalProblem, Task, Method
from unified_planning.io import PDDLReader
from unified_planning.io import PDDLWriter
from unified_planning.io.hpdl.hpdl_reader import HPDLReader
from unified_planning.io.hpdl.hpdl_writer import HPDLWriter
from unified_planning.engines.results import PlanGenerationResultStatus

# Register SIADEX
env = up.environment.get_env()
env.factory.add_engine('siadex', __name__, "SIADEXEngine")

Now, let's get a plan and the decomposition tree associated. We will use the HTN version of the Miconic domain, which represents the behaviour of an elevator that moves people between floors.

You can tell SIADEX to return the decomposition tree setting the parameter `decomposition_tree` to `True` while calling `OneShotPlanner()`

In [ ]:
# Get UPF problem
reader = HPDLReader()
problem = reader.parse_problem("./up-siadex/examples/ipc/Miconic/domain.hpdl", "./up-siadex/examples/ipc/Miconic/problem.hpdl")

# Plan
with env.factory.OneshotPlanner(name='siadex', params={'decomposition_tree': True}) as s:
    result = s.solve(problem)
    if result.status == PlanGenerationResultStatus.SOLVED_SATISFICING:
        print(f'{s.name()} found a valid plan!\n')
        print(f'The plan is:')
        for i,a in enumerate(result.plan.actions):
            print(f"{i}: {a}")
    else:
        print('No plan found!')

The decomposition tree is stored within the output of the `solve()` function

In [ ]:
dt = result.decomposition_tree
print(f'The decomposition tree for the given plan is:\n{dt}\n')

Looking at how much output it returns at first it may seem a complex structure, but it is not. The results is an object of the `DecompositionTree` class, that although has a bunch of functionality to play with, is mainly a structure that provides access to the different nodes in the tree. We will show the following things through this notebook:

1. How to extract nodes individually from the graph
2. Access to root and leaves nodes
2. Get unifications of a node
2. Different ways to plot the tree

But first, let's get a few statistics of the tree

In [ ]:
print(f'Maximum depth: {dt.depth}')
print(f'Actions: {dt.num_actions()}')
print(f'Tasks: {dt.num_tasks()}')

So, to solve our problem of the Miconic domain, we got a total of 20 actions, that are ultimatedly generated by 11 tasks, and the planner doesn't go lower than 6 levels in the HTN structure. Remember, tasks and actions can appear multiple times, even with the same parameters, because we are looking at an instantiation of the domain.

For example, we will see that the task `move`, which describes the decomposition schema that transports a person from one floor to another, is called repeatedly with different parameters.

Each element in the decomposition tree is of type `DecompositionTreeNode`, which provides detailed information of the node. Let's see an example:

In [ ]:
# Get root node
root = dt.root_nodes[0]
print(root)

There are two types of `DecompositionTreeNode`s: __tasks__ and __actions__. They are associated with the UPF `Task` and `Action` classes contained in the problem.

As a reminder, tasks are intermediate steps in the task network that produce the plan, and actions are (unsurprisingly) the primitives/actions being carried out in the plan.

Different attributes can be accessed depending of the node type. If it is a task, like we have seen before, the node will also contain the method chosen and the children (subtasks) that points into. Primitives on the other hand are the leaves of the tree, so they don't point anywhere.

In [ ]:
# Get some leaf node
leaf = dt.leaves_nodes[3]
print(leaf)

print(f'Is action? {leaf.is_action()}')

## Accessing nodes

There are multiple ways of accessing nodes in the decomposition tree. The `DecompositionTree` class provides simple methods to get roots (the upmost part of the tree, that is, the tasks/actions indicated in the tasks-goal), leaves (i.e. plan primitives), children of any node and so on.

Internally each node is identified by an unique integer ID (starting at 0) which can be used for indexing the tree. Because tasks can be called multiple times, even with the same parameters, keep in mind that the name is not enough to identify a node.

The ID of each node is the first number displayed when printing. You can also get them with `DecompositionTree.node_id(node: DecompositionTreeNode)` or `DecompositionTreeNode.id`

In [ ]:
node = dt.node(15) # Get node with ID=5
node

In [11]:
# Get second child
child_id = node.children[2]
# Alternatively
# child_id = dt.children(node.id)[2]

# Get node
child = dt.node(child_id)

print(child)
print("Child node ID:", dt.node_id(child))

19: action move_primitive
 - Depth: 3
 - Objects: f3 f1

Child node ID: 19


## Roots and leaves

Roots and leaves are particular nodes in the decomposition tree. Like we said before, roots are the tasks or actions specified in the task-goal (what we wanted to be solved), and leaves are the actions in the plan.

You can access directly to root and leaves nodes from the DT class

In [12]:
print(f'Root ids: {dt.root}')
print(dt.root_nodes)

print(f'\n\nLeaves ids: {dt.leaves}')
print(dt.leaves_nodes)

Root ids: [0]
[0: task solve_elevator
 - Method: solve_elevator_m1_go_ordering_0
 - Depth: 0
 - Objects: 
 - Children: 1 2
]


Leaves ids: [3, 4, 5, 6, 17, 18, 19, 20, 31, 32, 33, 34, 45, 46, 47, 48, 59, 60, 61, 62]
[3: action move_primitive
 - Depth: 2
 - Objects: f0 f0
, 4: action board_primitive
 - Depth: 2
 - Objects: p0 f0
, 5: action move_primitive
 - Depth: 2
 - Objects: f0 f1
, 6: action debark_primitive
 - Depth: 2
 - Objects: p0 f1
, 17: action move_primitive
 - Depth: 3
 - Objects: f1 f3
, 18: action board_primitive
 - Depth: 3
 - Objects: p2 f3
, 19: action move_primitive
 - Depth: 3
 - Objects: f3 f1
, 20: action debark_primitive
 - Depth: 3
 - Objects: p2 f1
, 31: action move_primitive
 - Depth: 4
 - Objects: f1 f2
, 32: action board_primitive
 - Depth: 4
 - Objects: p4 f2
, 33: action move_primitive
 - Depth: 4
 - Objects: f2 f1
, 34: action debark_primitive
 - Depth: 4
 - Objects: p4 f1
, 45: action move_primitive
 - Depth: 5
 - Objects: f1 f0
, 46: action board_primiti

Other think you could ask is: What tasks are at a given depth?

You can get that information with the following function

In [13]:
print(f"Nodes at depth 3: {dt.nodes_at_depth(3)}")

print(f"Nodes at depth 0: {dt.nodes_at_depth(0)}")

print(f"Nodes at maximum depth ({dt.depth}): {dt.nodes_at_depth(dt.depth)}")

Nodes at depth 3: [17, 18, 19, 20, 29, 30]
Nodes at depth 0: [0]
Nodes at maximum depth (6): [59, 60, 61, 62]


Note that at depth 0 is the root, but leaves can be at different depths, not necessarily at maximum depth

## Looking at the unifications

In case you wonder into which objects the parameters of an action or tasks are translated, you can look at `DecompositionTreeNode.unifications`.
It returns a dictionary of the UPF models `Parameter` -> `Object`, as stored in the UPF `Problem`.

If you are only interested in the objects, you can get them from `DecompositionTreeNode.objects`.

Let's say we want to retrieve the 2 object of the leaf node we kept earlier, and check for its type.

First, let show the tree node and the unifications

In [14]:
leaf

6: action debark_primitive
 - Depth: 2
 - Objects: p0 f1

In [15]:
unif = leaf.unifications

print(f"For action {leaf.name}")
for x,y in unif.items():
    print(f"- Parameter ({x}) gets instantiated by object ({y.type} {y})")

For action debark_primitive
- Parameter (person - object__compiled p) gets instantiated by object (person - object__compiled p0)
- Parameter (floor - object__compiled f) gets instantiated by object (floor - object__compiled f1)


We can index `unifications` with a `Parameter`

In [16]:
# Get Action
action = problem.action(leaf.name)

# Get Parameter
floor = action.parameters[1]

# Look for the object
obj = unif[floor]
print(f"Object {obj} of type {obj.type}")

Object f1 of type floor - object__compiled


Remember that if we define our problem directly from UPF we should already haver or Action and Parameters objects, so we can directly do:

```{python}
move = InstantaneousAction("move", l_from=Location, l_to=Location)
l_from = move.parameter("l_from")

# ...
# Define the rest of the problem and call the planner
# ...

unif = dt.node().unifications

obj = unif[floor]
print(f"Object {obj} of type {obj.type}")
```

Now, an example for a task:

In [17]:
unif = node.unifications

print(f"Node {node}")
for x,y in unif.items():
    print(f"- Parameter ({x}) gets instantiated by object ({y.type} {y})")

Node 15: task deliver_person
 - Method: deliver_person_m2_ordering_0
 - Depth: 2
 - Objects: p2 f3 f1
 - Children: 17 18 19 20

- Parameter (person - object__compiled p) gets instantiated by object (person - object__compiled p2)
- Parameter (floor - object__compiled o) gets instantiated by object (floor - object__compiled f3)
- Parameter (floor - object__compiled d) gets instantiated by object (floor - object__compiled f1)


## Plotting

By default the decomposition tree will get printed with the above format, but there are other functions to plot it in alternative ways.

In [18]:
print(dt.plot()) # All information

 |-0 solve_elevator()
   |-1 deliver_person(p0 - person - object__compiled, f0 - floor - object__compiled, f1 - floor - object__compiled)
     |-3 move_primitive(f0 - floor - object__compiled, f0 - floor - object__compiled)
     |-4 board_primitive(p0 - person - object__compiled, f0 - floor - object__compiled)
     |-5 move_primitive(f0 - floor - object__compiled, f1 - floor - object__compiled)
     |-6 debark_primitive(p0 - person - object__compiled, f1 - floor - object__compiled)
   |-2 solve_elevator()
     |-15 deliver_person(p2 - person - object__compiled, f3 - floor - object__compiled, f1 - floor - object__compiled)
       |-17 move_primitive(f1 - floor - object__compiled, f3 - floor - object__compiled)
       |-18 board_primitive(p2 - person - object__compiled, f3 - floor - object__compiled)
       |-19 move_primitive(f3 - floor - object__compiled, f1 - floor - object__compiled)
       |-20 debark_primitive(p2 - person - object__compiled, f1 - floor - object__compiled)
     |-16

In [19]:
print(dt.plot(names=False, objects=False)) # Only IDs

 |-0
   |-1
     |-3
     |-4
     |-5
     |-6
   |-2
     |-15
       |-17
       |-18
       |-19
       |-20
     |-16
       |-29
         |-31
         |-32
         |-33
         |-34
       |-30
         |-43
           |-45
           |-46
           |-47
           |-48
         |-44
           |-57
             |-59
             |-60
             |-61
             |-62
           |-58


There is also a compact version as was used in the IPC2020

In [20]:
print(dt.print_as_ipc())

3 move f0 f0
4 board p0 f0
5 move f0 f1
6 debark p0 f1
17 move f1 f3
18 board p2 f3
19 move f3 f1
20 debark p2 f1
31 move f1 f2
32 board p4 f2
33 move f2 f1
34 debark p4 f1
45 move f1 f0
46 board p1 f0
47 move f0 f3
48 debark p1 f3
59 move f3 f3
60 board p3 f3
61 move f3 f2
62 debark p3 f2
root 0
0 solve_elevator -> solve_elevator_m1_go_ordering_0 1 2
1 deliver_person p0 f0 f1 -> deliver_person_m2_ordering_0 3 4 5 6
2 solve_elevator -> solve_elevator_m1_go_ordering_0 15 16
15 deliver_person p2 f3 f1 -> deliver_person_m2_ordering_0 17 18 19 20
16 solve_elevator -> solve_elevator_m1_go_ordering_0 29 30
29 deliver_person p4 f2 f1 -> deliver_person_m2_ordering_0 31 32 33 34
30 solve_elevator -> solve_elevator_m1_go_ordering_0 43 44
43 deliver_person p1 f0 f3 -> deliver_person_m2_ordering_0 45 46 47 48
44 solve_elevator -> solve_elevator_m1_go_ordering_0 57 58
57 deliver_person p3 f3 f2 -> deliver_person_m2_ordering_0 59 60 61 62
58 solve_elevator -> solve_elevator_m1_abort_ordering_0 


## Testing PDDL parser TODO: Remove from Notebook

In [21]:
# !pip install --pre unified-planning[pyperplan]

In [22]:
# import unified_planning
# from unified_planning.shortcuts import *

In [23]:
# Location = UserType('Location')
# robot_at = unified_planning.model.Fluent('robot_at', BoolType(), l=Location)
# connected = unified_planning.model.Fluent('connected', BoolType(), l_from=Location, l_to=Location)
# move = unified_planning.model.InstantaneousAction('move', l_from=Location, l_to=Location)
# l_from = move.parameter('l_from')
# l_to = move.parameter('l_to')
# move.add_precondition(connected(l_from, l_to))
# move.add_precondition(robot_at(l_from))
# move.add_effect(robot_at(l_from), False)
# move.add_effect(robot_at(l_to), True)
# print(move)
# problem = unified_planning.model.Problem('robot')
# problem.add_fluent(robot_at, default_initial_value=False)
# problem.add_fluent(connected, default_initial_value=False)
# problem.add_action(move)
# NLOC = 10
# locations = [unified_planning.model.Object('l%s' % i, Location) for i in range(NLOC)]
# problem.add_objects(locations)
# problem.set_initial_value(robot_at(locations[0]), True)
# for i in range(NLOC - 1):
#     problem.set_initial_value(connected(locations[i], locations[i+1]), True)

# problem.add_goal(robot_at(locations[-1]))


In [24]:
# with OneshotPlanner(name='pyperplan') as planner:
#     result = planner.solve(problem)
#     if result.status == up.engines.PlanGenerationResultStatus.SOLVED_SATISFICING:
#         print("Pyperplan returned: %s" % result.plan)
#     else:
#         print("No plan found.")